In [ ]:
import pandas as pd
import matplotlib.ticker as ticker
import numpy as np
import matplotlib.pyplot as plt

#=========================================================================
#                        System parameters
#=========================================================================
m=39.948 *1.6747e-27 # Mass of Argon (atomic mass)(Avogadro's number) [kg]
epsilon=1.65e-21     # Minimum potential energy LJ [J]
sigma=3.4e-10        # Distance where the potential presents the maximum energy [m]
kB = 1.380e-23       # Boltzmann constant [J/K]
Na=6.022e23          # Avogadro's number

dt = 0.005           # integration time step
rc = 2.5             # LJ cutoff distance
rc2 = rc**2
rci = 1/rc
rc6i = rci**6
ec = 4.0*rc6i*(rc6i - 1)    # LJ cutoff energy
tau = 1                    # Berendsen coupling parameter

#=========================================================================
#                        Simulation functions
#=========================================================================

#--------------------- Generation of initial positions ------------------

def Pos_box_in(N,L): #Generates the box with N particles and their initial positions
    box_cell = int(np.floor(np.cbrt(N)))             # Divide the box into smaller boxes (cells)
    l_cell = float(L/box_cell)                       # length of each small cubic cell
    pos = np.zeros(shape=(N,3))                      # store coordinates
    for i in range(int(N//np.power(box_cell,3))+1):       # loop over big box, refill from start after first round filling
        for j in range(box_cell):                        # loop over x direction of the big box
            for k in range(box_cell):                    # loop over y direction of the big box
                for l in range(box_cell):                # loop over z direction of the big box

                    r = i*np.power(box_cell,3) + j*np.power(box_cell,2) + k*box_cell + l

                    if (i==1):                      # next round filling
                        if (r >= N):              # stop filling particle when exceeding given particle number
                            break

                    x_al = np.random.uniform(low=j*l_cell+0.1*l_cell, high=(j+1)*l_cell-0.1*l_cell)
                    y_al = np.random.uniform(low=k*l_cell+0.1*l_cell, high=(k+1)*l_cell-0.1*l_cell)
                    z_al = np.random.uniform(low=l*l_cell+0.1*l_cell, high=(l+1)*l_cell-0.1*l_cell)

                    pos[r] = [x_al,y_al,z_al]


    return pos

#--------------------- Periodic boundary conditions  ------------------

def PBC(pos,N,L): #Apply the periodic boundary conditions
    for i in range(N):
        pos[i] = pos[i] - L*np.floor(pos[i]/L)

    return pos

#--------------------- Generates initial velocities  ------------------

def V_i(N,T): #Generates the initial velocities of the particles
    vel = np.random.randn(N, 3)                     # velocity of all particles
    vel_mag = np.zeros(shape=(N,1))                 # speed of each particle
    sum_v = np.sum(vel, axis=0) / N                 # center of mass velocity
    sum_v2 = np.sum(vel**2) / N

    alpha = np.sqrt(3*T/(sum_v2))                   # scaling factor for velocity to satisfy MB

    for i in range(N):
        vel[i] = (vel[i] - sum_v)*alpha             # zero center of mass velocity

    for i in range (N):
        vel_mag[i] = np.sqrt(np.dot(vel[i],vel[i]))

    return vel, vel_mag

#--------------------- Calculation of Forces, Energy and Virial   ------------------

def Force_Energy_Virial(pos,N,L): #Obtain the force vector, the energy of the system and the virial coefficient of a configuration
    E = 0.0
    virial=0.0
    F = np.zeros(shape=(N,3))
    R = np.zeros(shape=(1,3))
    epsilon=1.65e-21
    sigma=3.4e-10

    for i in range(N-1):
        for j in range(i+1, N):

            R = pos[i] - pos[j]
            R = R - L*np.rint(R/L)      # PBC condition
            R2 = np.sum(R**2)

            if (R2 <= rc2 and R2>1e-15):
                r = np.sqrt(R2)
                r_inv = 1/r
                r2_inv = 1/R2
                r6_inv = r2_inv**3
                F_temp = 48*r2_inv*r6_inv*(r6_inv-0.5)*R
                E_temp = 4*r6_inv*(r6_inv-1)
                F_temp_mag=np.sqrt(np.sum(F_temp**2)) #Calculation of the magnitude of the force

                F[i] = F[i] + F_temp
                F[j] = F[j] - F_temp
                E = E + E_temp
                virial=virial+F_temp_mag*r  #Calculate the virial coefficient for this case

    return F, E, virial

#--------------------- Integration Method - Velocity Verlet    ------------------

def VelVerlet_Parameters(N,L,T,r_c,v_c,en,vir_c,f_c=None): #Integrate the equations of motion
    sumv2 = 0.0
    ek = 0.0
    etot = 0.0
    en_new = 0.0
    presion=0.0
    epsilon=1.65e-21
    sigma=3.4e-10
    kB = 1.380e-23

    #Energy calculation

    sumv2 = np.sum(v_c**2)
    temp = sumv2/(3*N)
    ek = 0.5*sumv2
    etot = en + ek

    #Calculation of pressure with the current virial coefficient

    volume=L**3
    preasure=(epsilon/sigma**3)*(  (N*T)/volume+vir_c*(1/(3*volume)) )/10.1325e3 #Calculate the pressure using the virial and convert to physical units

    #We apply Velocity Verlet
    r_new = r_c + v_c*dt + 0.5*f_c*dt**2       # r(t+dt)
    v_new = v_c + 0.5*f_c*dt                   # first update from f(t)
    f_new, en_new ,vir_new= Force_Energy_Virial(r_new,N,L) # new force f(t+dt)
    v_new = v_new + 0.5*f_new*dt               # second update of from f(t+dt)

    r_c = np.copy(r_new)
    v_c = np.copy(v_new)
    f_c = np.copy(f_new)

    #We apply Berdensen Thermostat
    v_c = v_c*np.sqrt(1+dt/tau*(T/temp-1))


    return r_c,v_c,f_c,en_new,temp,en,ek,etot,preasure,vir_new


#--------------------- Transcript of system configuration    ------------------

def Save_Configuration(r_c,N,L):
    r_PBC = PBC(r_c,N,L) 
    Config = []
    Config.append( str(N) ) #Create the string array
    for i in range(N): #Cycle to save r_c positions in .xyz format Config
        x = '{:.5f}'.format(r_PBC[i][0])
        y = '{:.5f}'.format(r_PBC[i][1])
        z = '{:.5f}'.format(r_PBC[i][2])
        pos_xyz =  str(x) + " " + str(y) + " " + str(z)
        Config.append(str(pos_xyz))

    return Config

#--------------------- Conversion of reduced units to physical units   ------------------

def Unit_Converter(Thermo_Par,N):

    Thermo_Par[:,0] = Thermo_Par[:,0]*dt*sigma*np.sqrt(m/epsilon)*1.0e12   # convert to ps
    Thermo_Par[:,1] = Thermo_Par[:,1]*epsilon/kB                           # convert to K
    Thermo_Par[:,2:] = Thermo_Par[:,2:]*epsilon*Na/N*1.0e-3                # convert to [kJ/mol]

    return Thermo_Par


In [ ]:
#=========================================================================
#                        Initial Conditions
#=========================================================================
np.random.seed(379245) #Ensures you always get the same data
N = 500 #Number of particles
L = 45  #Box length
T = 1  #Temperature
number_steps = 1500 #Number of steps in the simulation
density = m * N / (L * sigma)**3
spe_vol = 1/density

r_c = Pos_box_in(N,L)                        # get initial position
v_c, vs = V_i(N,T)                        # get initial velocity and speed
f_c, en , vir= Force_Energy_Virial(r_c,N,L)            # get initial force, vir es el coeficiente del virial
Configurations = []
Current_Configuration = Save_Configuration(r_c,N,L)
Configurations.append(Current_Configuration)                       # write initial trajectory

current_step = 0
Thermo_Par = []   # Save thermodynamical parameters
pressure=[]       #Save generated pressures

#=========================================================================
#                          Molecular Dynamics
#=========================================================================



data_general = {
    "Property": ["Particle number", "Box length (σ)", "Argon density (kg/m³)", "Specific Volume (m³/kg)"],
    "Value": [int(N), round(L, 1), round(density, 3), round(spe_vol,3)]
}

dg_frame = pd.DataFrame(data_general)

print("\nMolecular Dynamics Simulation \n")
print(dg_frame.to_string(index=False))

print("\n  \n")

#The simulation begins:

while current_step < number_steps:

    if (current_step%100 == 0):
        print ("Step %s of %s" % (current_step, number_steps))

    #Integration
    r_c,v_c,f_c,en_new,temp,en,ek,etot,press,vir_new = VelVerlet_Parameters(N,L,T,r_c,v_c,en,vir,f_c=f_c)

    if (current_step%10 == 0): #save every ten steps (can be modified)
        print(temp)
        pressure.append(press)
        Thermo_Par.append([current_step,temp,en,ek,etot])   # Store current thermo value
        Current_Configuration = Save_Configuration(r_c,N,L)
        Configurations.append(Current_Configuration)     #Stores the new configuration
        en = en_new
        vir=vir_new

    current_step = current_step + 1

Thermo_Par = np.asarray(Thermo_Par) #Arranges the data in an array of # of steps x thermodynamic parameter value
Thermo_Par = Unit_Converter(Thermo_Par,N) #Rewrites data into physical units

#=========================================================================
#                      Parameter Display
#=========================================================================

# Style configuration
tick_width = 2.5
tick_label_size = 14
frame_thickness = 2
xlabel_size = 18
ylabel_size = 18
title_size = 22

def Plot_Style(ax, ylabel=None, xlabel=None, titulo=None):
    if ylabel:
        ax.set_ylabel(ylabel, fontsize=ylabel_size)
    if xlabel:
        ax.set_xlabel(xlabel, fontsize=xlabel_size)
    if titulo:
        ax.set_title(titulo, fontsize=title_size)

    # Tick ​​and frame format
    ax.tick_params(axis='both', direction='in', which='both',
                   bottom=True, top=True, left=True, right=True,
                   width=tick_width, labelsize=tick_label_size)

    ax.spines['left'].set_linewidth(frame_thickness)
    ax.spines['bottom'].set_linewidth(frame_thickness)
    ax.spines['right'].set_linewidth(frame_thickness)
    ax.spines['top'].set_linewidth(frame_thickness)

    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

# ----- Temperature -----
fig1, ax1 = plt.subplots(figsize=(7, 5))
ax1.plot(Thermo_Par[:, 0], Thermo_Par[:, 1])
Plot_Style(ax1, ylabel='T(K)', xlabel='Time (ps)', titulo='Temperature')
plt.tight_layout()
plt.show()

# ----- Energy -----
fig2, ax2 = plt.subplots(figsize=(7, 5))
ax2.plot(Thermo_Par[:, 0], Thermo_Par[:, 2], label='V')
ax2.plot(Thermo_Par[:, 0], Thermo_Par[:, 3], label='T')
ax2.plot(Thermo_Par[:, 0], Thermo_Par[:, 4], label='E=T+V')
Plot_Style(ax2, ylabel='E (kJ/mol)', xlabel='Time (ps)', titulo='Energy')
ax2.legend(fontsize=14)
plt.tight_layout()
plt.show()

# ----- Pressure -----
fig3, ax3 = plt.subplots(figsize=(7, 5))
ax3.plot(Thermo_Par[:, 0], pressure,label='')
Plot_Style(ax3, ylabel='P(atm)', xlabel='Time (ps)', titulo='Pressure')
plt.tight_layout()
plt.show()


In [ ]:
#=========================================================================
#                       System Visualization
#=========================================================================

def Plot_Config(frame, elev, azim): #Plot the distribution of particles at a given time
    x_vals, y_vals, z_vals = [], [], []
    for line in frame[1:]:
        parts = line.strip().split()
        if len(parts) == 3:
            x, y, z = parts
            x_vals.append(float(x))
            y_vals.append(float(y))
            z_vals.append(float(z))

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_vals, y_vals, z_vals, s=20, color='blue')

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_title("System Configuration")
    ax.view_init(elev=elev, azim=azim)
    plt.show()


azim=0 #Azimuthal viewing angle
elev=0 #Elevation viewing angle
config_number=-1 # The system in the desired cycle step


P_Configuration = Configurations[config_number] #Configuration in the desired step
Plot_Config(P_Configuration, elev, azim) #Print the system configuration

print(P_Configuration)


In [ ]:
#=========================================================================
#                       Density Profile
#=========================================================================

def density_profile(configurations, L_reduced, N_bins, sigma, m):
    dz_reduced = L_reduced / N_bins
    V_bin_reduced = L_reduced**2 * dz_reduced  # volumen reducido del bin

    density_reduced = np.zeros(N_bins)

    for config in configurations[15:]:
        for particle in config[1:]:
            z= float(particle.split()[2])
            bin_index = int(z / dz_reduced)
            if bin_index >= N_bins:
                bin_index = N_bins - 1
            density_reduced[bin_index] += 1

    density_reduced /= (len(configurations[15:]) * V_bin_reduced)
    density_phys = density_reduced * (m / sigma**3)

    z_centers_reduced = np.linspace(dz_reduced/2, L_reduced - dz_reduced/2, N_bins)

    return z_centers_reduced, density_phys

N_bins = 30 #Number of bins (slices for calculate the density)
z_axis, rho_z = density_profile(Configurations, L, N_bins, sigma, m)

#Plot the density profile
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(z_axis, rho_z)
Plot_Style(ax, ylabel='Density [kg/m³]', xlabel='z [σ]', titulo='Density Profile along z-axis')
plt.tight_layout()
plt.show()
